### Attack Dectection For SPD

In [1]:
# import libs
import numpy as np
import pandas as pd
import os
import scipy.io as io
from sklearn import preprocessing
from scipy.stats import pearsonr
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from numpy import array
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
import math
from scipy.linalg import expm,logm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [2]:
## 计算黎曼距离
def riemann_distance(X,Y):
    X = np.reshape(X,(26,26))
    Y = np.reshape(Y,(26,26))
    gtype = 'L'
#     print(X.shape)
    if gtype == 'A':
        tmpEig = geneig(X,Y)
        tmp_dis = sum(mylog(tmpEig) ** 2)
    elif gtype == 'S':
        t = mylog(np.linalg.det(0.5 * (X + Y))) - 0.5 * (mylog(np.linalg.det(X))) + mylog(np.linalg.det(Y))
        if t == np.inf or np.isnan(t):
            eigX = np.linalg.eig(X)
            eigX = np.maximum(eigX,eps)
            eigY = np.linalg.eig(Y);
            eigY = np.maximum(eigY,eps)

            t = np.real(sum(mylog(np.linalg.eig(0.5*(X+Y))))) - 0.5 * (np.real(sum(mylog(eigX)))) + np.real(sum(mylog(eigY)))

        tmp_dis = t;
    elif gtype == 'J':
        tmp_dis = 0.5 * (np.dot(np.linalg.inv(Y),X)).trace() + 0.5 * (np.dot(np.linalg.inv(X),Y)).trace() - X.shape[0]
    elif gtype == 'L':
        tmp_dis = np.linalg.norm((logm(X) - logm(Y)),ord='fro') ** 2
    
    if tmp_dis <= 1e-15:
        dis = 0
    else:
        dis = np.sqrt(tmp_dis);
    return dis

## 计算广义特征值
def geneig(X,Y):
    w,v = np.linalg.eig(np.dot(np.linalg.inv(Y),X))
    return w

## 计算可能出现负数的对数函数
def mylog(X):
    CX = X + 0j
    return np.log(CX)

## 将协方差矩阵转成上三角矩阵
def dealSPD(A):
    n = A.shape[0]
    B = np.zeros(shape=(1, n*(n+1)//2))
    j = 0
    for i in range(0,n):
        B[:,j:j+n-i] = A[i,i:n]
        j = j + n -i
    return B

In [3]:
train_spd_X_path = '/mnt/hgfs/linuxfile/spd_train_npy_set/train_spd_X.npy'
train_spd_Y_path = '/mnt/hgfs/linuxfile/spd_train_npy_set/train_spd_Y.npy'
test_spd_X_path = '/mnt/hgfs/linuxfile/spd_test_npy_set/test_spd_X.npy'
test_spd_Y_path = '/mnt/hgfs/linuxfile/spd_test_npy_set/test_spd_Y.npy'

In [4]:
train_spd_X = np.load(train_spd_X_path)
train_spd_Y = np.load(train_spd_Y_path)
test_spd_X = np.load(test_spd_X_path)
test_spd_Y = np.load(test_spd_Y_path)

In [5]:
trainshape = train_spd_X.shape
testshape = test_spd_X.shape

In [6]:
trainshape

(10698, 26, 26)

In [7]:
train_spd_X_ = np.zeros(shape=(trainshape[0],trainshape[1] * (trainshape[2]+1) // 2))
test_spd_X_ = np.zeros(shape=(testshape[0],testshape[1] * (testshape[2]+1) // 2))

In [8]:
# for i in range(trainshape[0]):
#     train_spd_X_[i] = train_spd_X[i].ravel()
# for i in range(testshape[0]):
#     test_spd_X_[i] = test_spd_X[i].ravel()

In [9]:
for i in range(trainshape[0]):
    train_spd_X_[i] = dealSPD(train_spd_X[i])
for i in range(testshape[0]):
    test_spd_X_[i] = dealSPD(test_spd_X[i])

In [10]:
train_X = train_spd_X_
train_Y = train_spd_Y
test_X = test_spd_X_
test_Y = test_spd_Y

### 极限树

In [11]:
Extra_model = ExtraTreesClassifier(random_state = 1).fit(train_X,train_Y)

test_Y_hat = Extra_model.predict(test_X)

print(accuracy_score(test_Y, test_Y_hat))
print(precision_score(test_Y, test_Y_hat,average='micro'))
print(recall_score(test_Y, test_Y_hat,average='micro'))
print(f1_score(test_Y, test_Y_hat,average='micro'))
print(classification_report(test_Y, test_Y_hat))

/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


0.9761424731182796
0.9761424731182796
0.9761424731182796
0.9761424731182796
              precision    recall  f1-score   support

         0.0       0.95      1.00      0.98       242
         1.0       1.00      1.00      1.00       218
         2.0       0.87      0.91      0.89       246
         3.0       1.00      1.00      1.00       242
         4.0       0.91      0.87      0.89       247
         5.0       0.99      0.96      0.97       243
         6.0       0.99      0.99      0.99       249
         7.0       1.00      1.00      1.00       553
         8.0       1.00      0.99      0.99       242
         9.0       1.00      1.00      1.00       245
        10.0       1.00      1.00      1.00       249

    accuracy                           0.98      2976
   macro avg       0.97      0.97      0.97      2976
weighted avg       0.98      0.98      0.98      2976



普通方阵
0.9758064516129032
0.9758064516129032
0.9758064516129032
0.9758064516129032
              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97       242
         1.0       1.00      1.00      1.00       218
         2.0       0.87      0.92      0.89       246
         3.0       1.00      1.00      1.00       242
         4.0       0.92      0.86      0.89       247
         5.0       0.99      0.95      0.97       243
         6.0       0.99      0.99      0.99       249
         7.0       1.00      1.00      1.00       553
         8.0       1.00      0.99      1.00       242
         9.0       1.00      1.00      1.00       245
        10.0       1.00      1.00      1.00       249

    accuracy                           0.98      2976
   macro avg       0.97      0.97      0.97      2976
weighted avg       0.98      0.98      0.98      2976


### 随机森林

In [12]:
RandomForest_model = RandomForestClassifier().fit(train_X,train_Y)

test_Y_hat = RandomForest_model.predict(test_X)

print(accuracy_score(test_Y, test_Y_hat))
print(precision_score(test_Y, test_Y_hat,average='micro'))
print(recall_score(test_Y, test_Y_hat,average='micro'))
print(f1_score(test_Y, test_Y_hat,average='micro'))
print(classification_report(test_Y, test_Y_hat))

/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


0.9022177419354839
0.9022177419354839
0.9022177419354839
0.9022177419354839
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99       242
         1.0       1.00      0.99      1.00       218
         2.0       0.78      0.80      0.79       246
         3.0       1.00      1.00      1.00       242
         4.0       0.77      0.09      0.17       247
         5.0       0.99      0.95      0.97       243
         6.0       0.95      1.00      0.97       249
         7.0       1.00      1.00      1.00       553
         8.0       1.00      0.99      0.99       242
         9.0       0.54      1.00      0.70       245
        10.0       1.00      1.00      1.00       249

    accuracy                           0.90      2976
   macro avg       0.91      0.89      0.87      2976
weighted avg       0.92      0.90      0.88      2976



普通方阵
0.8780241935483871
0.8780241935483871
0.8780241935483871
0.8780241935483871
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96       242
         1.0       1.00      0.99      1.00       218
         2.0       0.91      0.53      0.67       246
         3.0       1.00      1.00      1.00       242
         4.0       0.50      0.07      0.13       247
         5.0       0.98      0.96      0.97       243
         6.0       0.96      0.99      0.98       249
         7.0       1.00      1.00      1.00       553
         8.0       1.00      0.99      0.99       242
         9.0       0.45      1.00      0.62       245
        10.0       1.00      1.00      1.00       249

    accuracy                           0.88      2976
   macro avg       0.88      0.87      0.85      2976
weighted avg       0.90      0.88      0.86      2976

### KNN

In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

In [14]:
# ## 交叉验证选择合适的K值
# k_range = range(1,32)
# k_error = []
# for k in k_range:
#     print('第' + str(k) + '次检验')
#     knn = KNeighborsClassifier(n_neighbors=k, metric=riemann_distance)
#     scores = cross_val_score(knn, train_X, train_Y, cv = 6, scoring='accuracy')
#     k_error.append(1-scores.mean())

In [15]:
# k_error

[0.03860534679379324,
 0.040474855113105335,
 0.03411852682744432,
 0.03533370723499729,
 0.03514675640306597,
 0.03477285473920366,
 0.035240231819031576,
 0.03673583847448125,
 0.03767059263413719,
 0.03841839596186214,
 0.038511871377827744,
 0.03944662553748357,
 0.03925967470555258,
 0.039914002617311595,
 0.03982052720134599,
 0.039914002617311595,
 0.03944662553748357,
 0.03916619928958687,
 0.03963357636941478,
 0.03897924845765566,
 0.03935315012151808,
 0.03925967470555258,
 0.03944662553748357,
 0.03916619928958687,
 0.03916619928958698,
 0.03916619928958687,
 0.039727051785380496,
 0.03925967470555236,
 0.03907272387362115,
 0.03907272387362115,
 0.03907272387362115,
 0.038698822209758954,
 0.03963357636941478,
 0.03897924845765566,
 0.0400074780332772,
 0.039914002617311706,
 0.0400074780332772,
 0.040474855113105335,
 0.04103570760889885,
 0.04131613385679567,
 0.04103570760889885,
 0.04122265844083006,
 0.040848756776967754,
 0.04131613385679567,
 0.04122265844083006,
 0.04094223219293325,
 0.04075528136100204,
 0.040287904281174014,
 0.04019442886520841,
 0.04038137969713962,
 0.04056833052907083,
 0.04075528136100204,
 0.04084875677696764,
 0.040848756776967754,
 0.040474855113105335,
 0.040287904281174125,
 0.040100953449242915,
 0.04019442886520841,
 0.03982052720134599,
 0.0400074780332772,
 0.03982052720134599,
 0.04019442886520841,
 0.039540100953449175]

In [16]:
knn_model = KNeighborsClassifier(n_neighbors=7).fit(train_X, train_Y)

/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [17]:
test_Y_hat = knn_model.predict(test_X)

In [18]:
print(accuracy_score(test_Y, test_Y_hat))
print(precision_score(test_Y, test_Y_hat,average='micro'))
print(recall_score(test_Y, test_Y_hat,average='micro'))
print(f1_score(test_Y, test_Y_hat,average='micro'))
print(classification_report(test_Y, test_Y_hat))

0.8410618279569892
0.8410618279569892
0.8410618279569892
0.8410618279569892
              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98       242
         1.0       0.70      0.98      0.82       218
         2.0       0.90      0.81      0.85       246
         3.0       0.33      0.05      0.09       242
         4.0       0.82      0.90      0.86       247
         5.0       0.51      0.98      0.67       243
         6.0       0.98      0.99      0.99       249
         7.0       1.00      0.76      0.87       553
         8.0       0.97      0.90      0.94       242
         9.0       1.00      0.97      0.98       245
        10.0       0.89      1.00      0.94       249

    accuracy                           0.84      2976
   macro avg       0.82      0.85      0.82      2976
weighted avg       0.84      0.84      0.82      2976



### 普通knn 普通方阵
0.8464381720430108
0.8464381720430108
0.8464381720430108
0.8464381720430108
              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98       242
         1.0       0.70      0.98      0.82       218
         2.0       0.93      0.81      0.87       246
         3.0       0.29      0.05      0.09       242
         4.0       0.82      0.93      0.87       247
         5.0       0.51      0.98      0.67       243
         6.0       0.98      0.99      0.99       249
         7.0       1.00      0.78      0.88       553
         8.0       0.97      0.90      0.94       242
         9.0       0.99      0.97      0.98       245
        10.0       0.94      1.00      0.97       249

    accuracy                           0.85      2976
   macro avg       0.83      0.85      0.82      2976
weighted avg       0.85      0.85      0.83      2976


### 逻辑回归

In [19]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(penalty='l2').fit(train_X, train_Y)
test_Y_hat = lr_model.predict(test_X)
print(accuracy_score(test_Y, test_Y_hat))
print(precision_score(test_Y, test_Y_hat,average='micro'))
print(recall_score(test_Y, test_Y_hat,average='micro'))
print(f1_score(test_Y, test_Y_hat,average='micro'))
print(classification_report(test_Y, test_Y_hat))

/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.34106182795698925
0.34106182795698925
0.34106182795698925
0.34106182795698925
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       242
         1.0       0.43      1.00      0.61       218
         2.0       0.00      0.00      0.00       246
         3.0       0.00      0.00      0.00       242
         4.0       0.00      0.00      0.00       247
         5.0       0.00      0.00      0.00       243
         6.0       0.00      0.00      0.00       249
         7.0       0.29      0.99      0.45       553
         8.0       0.00      0.00      0.00       242
         9.0       0.60      0.01      0.02       245
        10.0       0.40      1.00      0.58       249

    accuracy                           0.34      2976
   macro avg       0.16      0.27      0.15      2976
weighted avg       0.17      0.34      0.18      2976



/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedM

### GBDT

In [20]:
from sklearn.ensemble import GradientBoostingClassifier

GBDT_model = GradientBoostingClassifier(n_estimators=200).fit(train_X, train_Y)
test_Y_hat = GBDT_model.predict(test_X)

print(accuracy_score(test_Y, test_Y_hat))
print(precision_score(test_Y, test_Y_hat,average='micro'))
print(recall_score(test_Y, test_Y_hat,average='micro'))
print(f1_score(test_Y, test_Y_hat,average='micro'))
print(classification_report(test_Y, test_Y_hat))

/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.8672715053763441
0.8672715053763441
0.8672715053763441
0.867271505376344
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99       242
         1.0       1.00      1.00      1.00       218
         2.0       0.96      0.60      0.74       246
         3.0       1.00      0.76      0.87       242
         4.0       0.71      0.10      0.17       247
         5.0       1.00      0.99      0.99       243
         6.0       0.99      0.99      0.99       249
         7.0       1.00      0.99      0.99       553
         8.0       0.98      1.00      0.99       242
         9.0       0.40      1.00      0.57       245
        10.0       1.00      1.00      1.00       249

    accuracy                           0.87      2976
   macro avg       0.91      0.86      0.85      2976
weighted avg       0.92      0.87      0.86      2976



### SVM

In [21]:
from sklearn.svm import SVC
svm_model = SVC(kernel='linear', C = 1.0, random_state=1).fit(train_X, train_Y)

test_Y_hat = svm_model.predict(test_X)

print(accuracy_score(test_Y, test_Y_hat))
print(precision_score(test_Y, test_Y_hat,average='micro'))
print(recall_score(test_Y, test_Y_hat,average='micro'))
print(f1_score(test_Y, test_Y_hat,average='micro'))
print(classification_report(test_Y, test_Y_hat))

/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.938508064516129
0.938508064516129
0.938508064516129
0.938508064516129
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.97       242
         1.0       0.83      0.96      0.89       218
         2.0       0.78      0.96      0.86       246
         3.0       0.93      1.00      0.96       242
         4.0       0.84      0.74      0.78       247
         5.0       1.00      1.00      1.00       243
         6.0       1.00      1.00      1.00       249
         7.0       1.00      0.84      0.91       553
         8.0       1.00      1.00      1.00       242
         9.0       1.00      0.97      0.98       245
        10.0       0.99      1.00      1.00       249

    accuracy                           0.94      2976
   macro avg       0.94      0.95      0.94      2976
weighted avg       0.94      0.94      0.94      2976



### AdaBoost

In [22]:
from sklearn.ensemble import AdaBoostClassifier

AdaBoost_model = AdaBoostClassifier().fit(train_X, train_Y)
test_Y_hat = AdaBoost_model.predict(test_X)

print(accuracy_score(test_Y, test_Y_hat))
print(precision_score(test_Y, test_Y_hat,average='micro'))
print(recall_score(test_Y, test_Y_hat,average='micro'))
print(f1_score(test_Y, test_Y_hat,average='micro'))
print(classification_report(test_Y, test_Y_hat))

/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.4230510752688172
0.4230510752688172
0.4230510752688172
0.42305107526881724
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       242
         1.0       1.00      1.00      1.00       218
         2.0       0.00      0.00      0.00       246
         3.0       0.00      0.00      0.00       242
         4.0       0.00      0.00      0.00       247
         5.0       0.00      0.00      0.00       243
         6.0       0.13      1.00      0.23       249
         7.0       1.00      0.98      0.99       553
         8.0       0.00      0.00      0.00       242
         9.0       0.00      0.00      0.00       245
        10.0       1.00      1.00      1.00       249

    accuracy                           0.42      2976
   macro avg       0.28      0.36      0.29      2976
weighted avg       0.35      0.42      0.36      2976



/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

### 决策树

In [23]:
from sklearn.tree import DecisionTreeClassifier

DecisionTree_model = DecisionTreeClassifier(random_state=3).fit(train_X, train_Y)
test_Y_hat = DecisionTree_model.predict(test_X)

print(accuracy_score(test_Y, test_Y_hat))
print(precision_score(test_Y, test_Y_hat,average='micro'))
print(recall_score(test_Y, test_Y_hat,average='micro'))
print(f1_score(test_Y, test_Y_hat,average='micro'))
print(classification_report(test_Y, test_Y_hat))

0.8182123655913979
0.8182123655913979
0.8182123655913979
0.8182123655913978
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99       242
         1.0       1.00      0.99      1.00       218
         2.0       0.00      0.00      0.00       246
         3.0       1.00      1.00      1.00       242
         4.0       0.00      0.00      0.00       247
         5.0       0.97      0.96      0.96       243
         6.0       0.84      0.95      0.89       249
         7.0       1.00      0.97      0.98       553
         8.0       0.87      0.99      0.93       242
         9.0       0.35      0.99      0.52       245
        10.0       1.00      1.00      1.00       249

    accuracy                           0.82      2976
   macro avg       0.73      0.80      0.75      2976
weighted avg       0.75      0.82      0.77      2976



### 黎曼度量 KNN

In [24]:
knn_model = KNeighborsClassifier(n_neighbors=7, metric=riemann_distance).fit(train_X, train_Y)

/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [25]:
test_spd_Y_hat = knn_model.predict(test_X)

ValueError: cannot reshape array of size 351 into shape (26,26)

In [ ]:
print(accuracy_score(test_Y, test_spd_Y_hat))
print(precision_score(test_Y, test_spd_Y_hat,average='micro'))
print(recall_score(test_Y, test_spd_Y_hat,average='micro'))
print(f1_score(test_Y, test_spd_Y_hat,average='micro'))
print(classification_report(test_Y, test_spd_Y_hat))

A riedis

0.9616935483870968

0.9616935483870968

0.9616935483870968

0.9616935483870968

          precision    recall  f1-score   support

     0.0       0.92      0.99      0.95       242
     1.0       0.87      1.00      0.93       218
     2.0       0.89      0.91      0.90       246
     3.0       1.00      1.00      1.00       242
     4.0       0.90      0.88      0.89       247
     5.0       0.98      0.99      0.99       243
     6.0       0.99      0.97      0.98       249
     7.0       1.00      0.94      0.97       553
     8.0       1.00      0.98      0.99       242
     9.0       1.00      0.96      0.98       245
    10.0       1.00      1.00      1.00       249

accuracy                           0.96      2976


macro avg 0.96 0.97 0.96 2976 weighted avg 0.96 0.96 0.96 2976


## J
0.9637096774193549
0.9637096774193549
0.9637096774193549
0.9637096774193549
              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96       242
         1.0       0.91      1.00      0.95       218
         2.0       0.87      0.91      0.89       246
         3.0       1.00      1.00      1.00       242
         4.0       0.90      0.86      0.88       247
         5.0       0.98      0.99      0.98       243
         6.0       0.99      0.97      0.98       249
         7.0       1.00      0.96      0.98       553
         8.0       1.00      0.97      0.98       242
         9.0       1.00      0.96      0.98       245
        10.0       1.00      1.00      1.00       249

    accuracy                           0.96      2976
   macro avg       0.96      0.96      0.96      2976
weighted avg       0.96      0.96      0.96      2976